In [1]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
import numpy as np
from custom_dataset import Binary_Dataset
from matplotlib import pyplot as plt
from PIL import Image
import os
from inception import inception_v3
from Trainer import train_loop
from torch.optim import Adam
import pandas as pd

%matplotlib inline
torch.cuda.empty_cache()

# Define Dataloaders for each class


In [2]:
#define relevant paths
labels_dir="Data\list_attr_celeba.csv"
image_dir="Data\img_align_celeba\img_align_celeba"

############################################define batch size for all data loaders here
bathsize=1

##############################################define num workers for data laoders here
workers=4

#define composed transforms
composed = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize([299,299]),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])


labels=["Male","Smiling","Wavy_Hair"]
label_number=1
# #Choose which label 
# ds=Binary_Dataset(labels_dir, labels[label_number],image_dir ,transform=composed, partition="Train")
# dataset_loader=DataLoader(ds,batch_size=bathsize, shuffle=True, num_workers=workers)

# ds_val=Binary_Dataset(labels_dir, labels[label_number],image_dir ,transform=composed, partition="Val")
# val_loader=DataLoader(ds_val,batch_size=bathsize, shuffle=True, num_workers=workers)

"""
# male
ds1=Binary_Dataset(labels_dir, labels[0],image_dir ,transform=composed)
dataset_loader=DataLoader(ds1,batch_size=bathsize, shuffle=True, num_workers=workers)

# Smiling
ds2=Binary_Dataset(labels_dir, labels[1],image_dir ,transform=composed)
dataset_loader=DataLoader(ds2,batch_size=bathsize, shuffle=True, num_workers=workers)

# wavy hair
ds3=Binary_Dataset(labels_dir, labels[2],image_dir ,transform=composed)
dataset_loader=DataLoader(ds3,batch_size=bathsize, shuffle=True, num_workers=workers)

# ???????????????
ds4=Binary_Dataset(labels_dir, labels[3],image_dir ,transform=composed)
dataset_loader=DataLoader(ds4,batch_size=bathsize, shuffle=True, num_workers=workers)
"""

"""
for idx, (data, image) in enumerate(ds):
    print(idx)
"""


# data=next(iter(dataset_loader))
# sample, target = data
# #print(sample)
# print(target)


'\nfor idx, (data, image) in enumerate(ds):\n    print(idx)\n'

# Define CNN's

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model=inception_v3().to(device)

C:\Users\whitt\Documents\UH\Fall 2020\CIVE 6397\Final Project\DeepLearning_Project-pytorch_style\DeepLearning_Project-pytorch_style\inception.py:81: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn('The default weight initialization of inception_v3 will be changed in future releases of '


In [4]:
model.load_state_dict(torch.load('Smiling_12_8_4epochs_extractionline.pt'))
model.eval()

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

# Training Loop

In [5]:
# learning_rate=0.0001
# save_name="Smiling_12_8"
# optimizer = Adam(params=model.parameters(), lr=learning_rate)
# train_loop(model=model,dataloader=dataset_loader,epochs=5,optimizer=optimizer,save_name="Smiling_12_8")

In [6]:
#check validation accuracy
#check feature extrcation

# 162770
# 19867
# 19962

bathsize = 50
FE_ds=Binary_Dataset(labels_dir, labels[label_number],image_dir ,transform=composed, partition="Val")
FE_loader=DataLoader(FE_ds,batch_size=bathsize, shuffle=False, num_workers=workers)

device = 'cuda'

FE_data=iter(FE_loader)

#model=model.to(device)

model.eval()

correct=0
tested_samples=0
limit=10


l = ['label']
range_int = range(1,2049)
r = [str(x) for x in range_int]
col = l + r   
px = pd.DataFrame(columns = col)  
px.to_csv('val_img.csv', mode='a', index=False, header=True)
    
    
for data in FE_data:
    sample, target = data
    
    #send to device
    sample=sample.to(device)
    target=target.to(device)
    
    output, extraction = model(sample)
    
    print(output.shape)
    print(extraction.shape)
    print(extraction)
    
    #rediction, extraction = output
        
    target_val = target.cpu().numpy()
    label_val = target_val[0]
        
    extract = extraction.cpu().numpy()
            
    data = np.insert(extract, 0, label_val, axis = None)
    data = np.reshape(data, (50, 2049))
 
    px = pd.DataFrame(data)   
    px.to_csv('val_img.csv', mode='a', index=False, header=False)   
        
    
    pred=torch.argmax(output,dim=1,keepdim=True)

    correct+=torch.eq(pred,target.view_as(pred)).sum()
    tested_samples+=pred.size()[0]
    if tested_samples>limit:
        break

    print(correct)

#print precision
print(correct.item()/tested_samples)

tensor([[2.7961e-01, 1.7455e-01, 4.8970e-02,  ..., 3.7502e-01, 2.1263e-03,
         0.0000e+00],
        [2.7180e-01, 1.3757e-01, 1.9916e-02,  ..., 9.1689e-02, 6.5760e-03,
         2.6529e-03],
        [1.8935e-01, 1.3527e-01, 2.2184e-02,  ..., 2.7001e-01, 6.1071e-04,
         1.8941e-03],
        ...,
        [1.7692e-01, 1.7051e-01, 8.4418e-02,  ..., 3.3001e-01, 0.0000e+00,
         0.0000e+00],
        [8.9386e-02, 1.4086e-01, 5.6014e-02,  ..., 2.5259e-01, 2.2859e-04,
         0.0000e+00],
        [1.9029e-01, 2.5159e-01, 1.0463e-01,  ..., 1.5727e-01, 1.3610e-03,
         1.0420e-03]], device='cuda:0')
torch.Size([50, 2])
torch.Size([50, 2048])
tensor([[2.7961e-01, 1.7455e-01, 4.8970e-02,  ..., 3.7502e-01, 2.1263e-03,
         0.0000e+00],
        [2.7180e-01, 1.3757e-01, 1.9916e-02,  ..., 9.1689e-02, 6.5760e-03,
         2.6529e-03],
        [1.8935e-01, 1.3527e-01, 2.2184e-02,  ..., 2.7001e-01, 6.1071e-04,
         1.8941e-03],
        ...,
        [1.7692e-01, 1.7051e-01, 8.4418e

ValueError: cannot reshape array of size 102401 into shape (50,2049)

In [ ]:
#torch.save(model.state_dict(),save_name+".pt")  

In [ ]:
# df_attr_train=pd.read_csv(labels_dir, nrows = 162770, usecols=["image_id",labels[label_number]]) 
# df_attr_val=pd.read_csv(labels_dir, skiprows = [i for i in range(1,162771)], nrows = 19867, usecols=["image_id",labels[label_number]])
# df_attr_test=pd.read_csv(labels_dir, skiprows = [i for i in range(1,182638)], nrows = 19962, usecols=["image_id",labels[label_number]])

# #change -1 to 0 in csv, remove this if not needed
# df_attr_train.replace(-1,0,inplace=True)
# df_attr_val.replace(-1,0,inplace=True)
# df_attr_test.replace(-1,0,inplace=True)

df_attr_train=pd.read_csv(labels_dir, nrows = 162770, usecols=["image_id"]) 
df_attr_val=pd.read_csv(labels_dir, skiprows = [i for i in range(1,162771)], nrows = 19867, usecols=["image_id"])
df_attr_test=pd.read_csv(labels_dir, skiprows = [i for i in range(1,182638)], nrows = 19962, usecols=["image_id"])

train_imgs = df_attr_train.to_numpy()
val_imgs = df_attr_val.to_numpy()
test_imgs = df_attr_test.to_numpy()

In [ ]:
df_attr_train.info()

In [ ]:
traindata = pd.read_csv('train_img.csv')
traindata.insert(0, 'img_id', train_imgs)
traindata.to_csv('train.csv', mode='a', index=False)

valdata = pd.read_csv('val_img.csv')
valdata.insert(0, 'img_id', val_imgs)
valdata.to_csv('val.csv', mode='a', index=False)

testdata = pd.read_csv('test_img.csv')
testdata.insert(0, 'img name', test_imgs)
testdata.to_csv('test.csv', mode='a', index=False)